# Covid 19 Insights and Analytics

![][some-id]

[some-id]: https://images.unsplash.com/photo-1584483766114-2cea6facdf57?q=80&w=2070&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D "Python Logo"

# Data Collection

We have collected the dataset from the publicly available dataset as provided by WHO(World Health Organization) for Covid 19 confirmed cases, death counts, and daily updated dataset. The dataset are fetched from the Github repositories freely available to everyone. The links mainly are:
- [Link 1 - Confirmed Cases](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv)
- [Link 2 - Deaths Globally](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv)
-  [Link 3 - Twitter Data](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv)
- [Link 4 - Global Vaccination](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv)

In [3]:

# Initial Imports
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import chart_studio.plotly as py
import chart_studio.tools as tls
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

import plotly.express as px
import folium
from IPython.display import IFrame

import pymongo
from pymongo import MongoClient
from sqlalchemy import create_engine

%matplotlib inline
warnings.filterwarnings("ignore")

from wordcloud import WordCloud, ImageColorGenerator
from plotly.offline import iplot
import nltk
from nltk.tokenize import RegexpTokenizer

import nltk
import re
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Mongo DB connection

In [4]:
client = MongoClient("localhost",27017)

In [5]:
db = client.db_dap2
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'db_dap2')

In [7]:
#reading 3 files and storing them
dataset1 = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
dataset2 = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
dataset3 = pd.read_csv('data/covid19_tweets.csv')
dataset4 = pd.read_json('data/global.json')
dataset3.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False


In [8]:
dataset1_dict = dataset1.to_dict(orient="records")
dataset2_dict = dataset2.to_dict(orient="records")
dataset3_dict = dataset3.to_dict(orient="records")
dataset4_dict = dataset4.to_dict(orient="records")

In [ ]:
db.collection1.insert_many(dataset1_dict)
db.collection2.insert_many(dataset2_dict)
db.collection3.insert_many(dataset3_dict)
db.collection4.insert_many(dataset4_dict)

In [ ]:
print(list(db.collection3.find().limit(1)))

In [ ]:
collection1 = db["collection1"]
r1 = collection1.find()
conf_cases_df = pd.DataFrame(r1)

collection2 = db["collection2"]
r2 = collection2.find()
death_cases_df = pd.DataFrame(r2)

collection3 = db["collection3"]
r3 = collection3.find()
twitter = pd.DataFrame(r3)

collection4 = db["collection4"]
r4 = collection3.find()
vaccination_df = pd.DataFrame(r4)

In [ ]:
def drop_id(dataframes):
    for i in dataframes:
        i.drop('_id',axis=1,inplace=True)
drop_id([conf_cases_df,
         death_cases_df,twitter
])

In [ ]:
# Confirmed Cases of COVID 19
conf_cases_df.head(5)

In [ ]:
# Death Cases of Covid19
death_cases_df.head(5)

In [ ]:
# Twitter Tweets over coronavirus
twitter.head()

In [ ]:
# Fetching the Vaccination data from the semistructured dataset as stored in MongoDB
# Connect to MongoDB
client = MongoClient('localhost', 27017)  # Update with your MongoDB connection details
db = client['db_dap2']  # Replace 'your_database_name' with your actual database name
collection = db['collection4']  # Replace 'your_collection_name' with your actual collection name

# Initialize empty lists to store extracted information
country_names = []
regions = []
income_levels = []
country_codes = []
populations = []
has_vaccines = []
income_level_name = []
response_date = []

# Data Retrival from MongoDB
cursor = collection.find()
for country_info in cursor:
    fields = country_info.get('fields', {})
    country_names.append(fields.get('countryName'))
    regions.append(fields.get('wbRegion'))
    income_levels.append(fields.get('wbIncomeLevelName'))
    country_codes.append(fields.get('wbCountryCode'))
    populations.append(fields.get('wbPopulation2019'))
    has_vaccines.append(fields.get('owidHasVaccine', False))
    income_level_name.append(fields.get('wbIncomeLevelName'))
    response_date.append(fields.get('mostRecentResponseDate'))

# DataFrame creation
vaccination_df = pd.DataFrame({'CountryName': country_names,
                                'Region': regions,
                                'IncomeLevel': income_levels,
                                'CountryCode': country_codes,
                                'Population': populations,
                                'HasVaccine': has_vaccines,
                                'IncomeLevelName': income_level_name,
                                'ResponseDate': response_date})

vaccination_df

### Checking the timeline

In [ ]:
# Lets grab the dates columns only for now. 

death_cases_df.columns[4:]

### Null Values Checkpoint

In [ ]:
# Now let's check the sum of the deaths for a particular country
death_cases_df.isna().sum()

In [ ]:
death_cases_df.dropna(subset=['Lat'],inplace=True)

In [ ]:
conf_cases_df.dropna(subset=['Lat'],inplace=True)

In [ ]:
conf_cases_df.isna().sum()

# Postgres Integration
Now we connect and save the Datasets to Postgres

In [ ]:
db_params = {
    "host": "localhost",
    "database": "dap",
    "user": "postgres",
    "password": "3679",
}
import psycopg2
conn = psycopg2.connect(**db_params)

In [ ]:
cursor = conn.cursor()
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}/{db_params["database"]}')

In [ ]:
df1 = conf_cases_df.copy()
df2 = death_cases_df.copy()
df3 = twitter.copy()
df4 = vaccination_df.copy()

In [ ]:
dataframes = [df1,df2,df3,df4]
table_names = ["c1","c2","c3","c4"]

# Convert ObjectId to string in each DataFrame

# Insert data into PostgreSQL tables
for df, table_name in zip(dataframes, table_names):
    df.to_sql(table_name, engine, if_exists='replace', index=False)

# Commit the transaction
conn.commit()
# Close the connection
#conn.close()

In [ ]:
### Fetching from postgres

In [ ]:
dict1 = {}
def fetch(tables):
    count=4
    for i in tables:
        conn = psycopg2.connect(**db_params)
        query = (f"SELECT * FROM {i};")
        for i in range(count,0,-1):
            dict1[count] = pd.read_sql_query(query, conn)
            count-=1
            break
        # Display the DataFrame
        
        
fetch(['c1','c2','c3','c4'])

In [ ]:
conf_cases_df = dict1[4]
death_cases_df = dict1[3]
twitter = dict1[2]
vaccination_df = dict1[1]

In [ ]:
conf_cases_df.head(5)

In [ ]:
death_cases_df.head(5)

# EDA - Visualizations -> Geographical Analytics Checkpoint

In [ ]:
# folium map depicting the number of COVID cases all over the world
m = folium.Map(location=[death_cases_df['Lat'].mean(), death_cases_df['Long'].mean()], zoom_start=4)

for index, row in conf_cases_df.iterrows():
    folium.CircleMarker(location=[row['Lat'],row['Long']],
                       radius=row['3/9/23']/1000000,
                       color = 'Green',
                       fill = True,
                       fill_opacity = 0.15,
                       popup = f"{row['Country/Region']}: {row['3/9/23']} deaths").add_to(m)
    
m.save('covid_deaths_map.html')

IFrame(src='./covid_deaths_map.html', width=1700, height=600)   

In [ ]:
# Country wise color distributed death count representation over the world
fig = px.choropleth(death_cases_df, locations="Country/Region", locationmode = "country names",
                    color="3/9/23",
                    hover_name="Country/Region",
                    scope = 'world',
                    template = 'plotly_dark',
                    color_continuous_scale="Blues")
fig.show()

In [ ]:
# Country wise color distributed death count representation over asia
fig = px.choropleth(death_cases_df, locations="Country/Region", locationmode = "country names",
                    color="3/9/23",
                    hover_name="Country/Region",
                    scope = 'asia',
                    template = 'plotly_dark',
                    color_continuous_scale="Viridis")
fig.show()

In [ ]:
# Country wise color distributed death count representation over Europe

fig = px.choropleth(death_cases_df, locations="Country/Region", locationmode = "country names",
                    color="3/9/23",
                    hover_name="Country/Region",
                    scope = 'europe',
                    template = 'plotly_dark',
                    color_continuous_scale="Blues")
fig.show()

In [ ]:
# Country wise color distributed death count representation over North America

fig = px.choropleth(death_cases_df, locations="Country/Region", locationmode = "country names",
                    color="3/9/23",
                    hover_name="Country/Region",
                    scope = 'north america',
                    template = 'plotly_dark',
                    color_continuous_scale="Greens")
fig.show()

In [ ]:
# Country wise color distributed death count representation over South America

fig = px.choropleth(death_cases_df, locations="Country/Region", locationmode = "country names",
                    color="3/9/23",
                    hover_name="Country/Region",
                    scope = 'south america',
                    template = 'plotly_dark',
                    color_continuous_scale="Greens")
fig.show()

## Dataframe Manipulation
> Now here we wanted to show the increase in the deaths overtime for each and every country with the help of a animation code using plotly. However, our code wasn't able to do so as the data is in a format where the dates are in the column names. The plotly arguments take the date as a instance value present in the dataframe. So we need to fetch the date data into the dataframe. 
>> We also need to keep in mind the length of the data as well. The dates that we have are almost above thousands of them ranging from 2020 to 2023. So to create a melted dataframe for each and every date would create a very large dataset that would be difficult to work with. So skipping a few dates might be a good idea when our target is to show the progress/increase in deaths over time. 

In [ ]:
# Checking the dates in the columns here once for idea
list(death_cases_df.columns[4:15]),len(list(death_cases_df.columns[4:]))

**So the count of date elements present in our dataframe is *1143***
> **That means 1143 days**

In [ ]:
from datetime import datetime
date_objects = [datetime.strptime(date_str, '%m/%d/%y') for date_str in list(death_cases_df.columns[4:])]

In [ ]:
len(date_objects),len(death_cases_df)

> ### Headnote for the upcoming task
Ok now to show the animation of the scatter plot changing over time with the deaths accumulated for the given time period we need to change the dataset a bit. we are not going to change the actual data. however make a copy of the dataframe and then add rows into it as per given date columns. 

In [ ]:
# Earlier we made the date objects so as to know how many dates we are working with. since
# we are working with a lot of data we need to cut short the dates by keeping dates with a 30 days gap.

In [ ]:
len(date_objects[::10])

In [ ]:
animation_death_df = pd.melt(death_cases_df, id_vars=['Province/State','Country/Region','Lat','Long'], var_name='Date', value_name='Deaths')

In [ ]:
animation_death_df[animation_death_df['Country/Region']=='Afghanistan'].head(5)

In [ ]:
columns_to_take = list(death_cases_df.columns[:4])
for i in list(death_cases_df.columns[4::]):
    columns_to_take.append(i)
columns_to_take[:10]

In [ ]:
for_animation = death_cases_df[columns_to_take].copy()
for_animation['Total Death'] = death_cases_df['3/9/23']
for_animation.head(5)

In [ ]:
# Here we remove all the columns except the columns that have the dates
new_columns_to_take = list(for_animation.columns)
new_columns_to_take = new_columns_to_take[4:]
new_columns_to_take[:10]

In [ ]:
# for_animation = for_animation.drop('Province/State',axis=1)
agg_dict = {col:'sum' for col in new_columns_to_take}
agg_dict['Lat'] = 'first'
agg_dict['Long'] = 'first'
for_animation = for_animation.groupby('Country/Region').agg(agg_dict).reset_index()
for_animation.head()

In [ ]:
new_columns_to_take = list(for_animation.columns[:1])
for i in list(for_animation.columns[1:-3:10]):
    new_columns_to_take.append(i)
new_columns_to_take.append('Lat')
new_columns_to_take.append('Long')
new_columns_to_take.append('Total Death')
print(new_columns_to_take[1:4],new_columns_to_take[-2:],len(new_columns_to_take))
for_animation = for_animation[new_columns_to_take]
for_animation.head(5)

In [ ]:
animation_death_df = pd.melt(for_animation, id_vars=['Country/Region','Lat','Long','Total Death'], var_name='Date', value_name='Deaths')
animation_death_df['Date'] = pd.to_datetime(animation_death_df['Date']).dt.date
animation_death_df = animation_death_df.sort_values(by=['Country/Region','Date'])

animation_death_df.head(5)

Well our Dataset is now ready! Lets head over to the animation code!
### Scatter plot over time showing the increasing death count.

In [ ]:
px.scatter(data_frame=animation_death_df,
          x = 'Deaths',
          y = 'Total Death',
          size='Deaths',
          color='Country/Region',
          title='COVID19 Global Deaths Analytics 2020-2023',
          labels={'Deaths':'Total Deaths till Date',
                  'Lat':'Latitude'},
          log_x=False,log_y=True,
          hover_name = 'Country/Region',
          animation_frame='Date',
          range_x=[-500,1300000],
          range_y=[40000,2400000],
          size_max=20*3)